In [ ]:
#В промежуточную выгрузку добавить телефон!!!

import pandas as pd
import numpy as np
import re
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import datetime as dt
#NOW = dt.datetime(2019,8,14)

In [ ]:
NOW = dt.datetime.today()
NOW

In [ ]:
orders_rfm = pd.read_csv('first_block_ext.csv', sep=';')
# orders_4_1 = pd.read_csv('Выгрузка по чекам 4_1.csv', header=None, encoding='utf-8', sep=';', low_memory=False)
# orders_4_2 = pd.read_csv('Выгрузка по чекам 4_1.csv', header=None, encoding='utf-8', sep=';', low_memory=False)

In [ ]:
#orders_rfm.iloc[:, 5] = orders_rfm.iloc[:, 5].astype(dtype='Int64')

In [ ]:
orders_rfm.tail()

In [ ]:
orders_rfm.info()

In [ ]:
# Make the date_placed column datetime
orders_rfm['Дата'] = pd.to_datetime(orders_rfm['Дата'])

In [ ]:
orders_rfm.head()

In [ ]:
orders_rfm.info()

In [ ]:
# Подготавливаем данные непосредственно для агоритма RFM
#rfmTable = orders_rfm.groupby(['Владелец карты', 'Дата']).agg({'Дата': lambda x: (NOW - x.max()).days, # Recency
#                                        'ID': lambda x: len(x),      # Frequency
#                                        'Сумма': lambda x: x.sum()}) # Monetary Value


In [ ]:
# Работаем с ext вариантом
rfmTable = orders_rfm.groupby(['Владелец карты', 'Подразделение', 'Телефон', 'Дата']).agg({'Дата': lambda x: (NOW - x.max()).days, # Recency
                                        'ID': lambda x: len(x),      # Frequency
                                        'Сумма': lambda x: x.sum()}) # Monetary Value


In [ ]:
rfmTable.head(50)

In [ ]:
rfmTable1 = rfmTable.add_suffix('_Count').reset_index()

In [ ]:
rfmTable1

In [ ]:
# Удаление пробелов
rfmTable1['Владелец карты'] = rfmTable1['Владелец карты'].str.lstrip()

In [ ]:
rfmTable1.info()

In [ ]:
rfmTable2 = rfmTable1.groupby(['Владелец карты']).agg({'Дата_Count': lambda x: x.min(), # Recency
                                        'ID_Count': lambda x: len(x),      # Frequency
                                        'Сумма_Count': lambda x: x.sum()}) # Monetary Value


In [ ]:
rfmTable2.head(50)

In [ ]:
# rfmTable3 = rfmTable2.reset_index()

In [ ]:
# rfmTable3

In [ ]:
rfmTable2.info()

In [ ]:
rfmTable2.rename(columns={'Дата_Count': 'recency', 
                         'ID_Count': 'frequency', 
                         'Сумма_Count': 'monetary_value'}, inplace=True)

In [ ]:
rfmTable2

In [ ]:
# Проверка
rfmTable1.iloc[1, 0]

In [ ]:
pens = rfmTable1[rfmTable1.iloc[:, 0]=='Таиркина Татьяна Николаевна']
pens

In [ ]:
rfmTable1.info()

In [ ]:
rfmTable1.shape

In [ ]:
quantiles = rfmTable2.quantile(q=[0.25,0.5,0.75])

In [ ]:
quantiles

In [ ]:
quantiles = quantiles.to_dict()

In [ ]:
quantiles

In [ ]:
rfmSegmentation = rfmTable2

In [ ]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        print (x)
        print (d[p][0.25])
        print (d)
        print (p)
        print(" ")
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [ ]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [ ]:
rfmSegmentation['RFMClass'] = (rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str))

In [ ]:
rfmSegmentation.head()

In [ ]:
rfmSegmentation.index

In [ ]:
rfmSegmentation.to_csv('rfm-table_mark.csv', sep=';')

In [ ]:
rfmSegmentation[rfmSegmentation['RFMClass']=='211'].sort_values('monetary_value', ascending=False).head(100)

In [ ]:
rfmSegmentation.info()

In [ ]:
rfmSegmentation[rfmSegmentation.recency<=60]     # rfmSegmentation[rfmSegmentation['RFMClass']=='111']

In [ ]:
rfmSegmentation

In [ ]:
rfmSegmentation.index